Tarea 1: creación de bases de datos

In [1]:
import sqlite3
from faker import Faker
import os
import random

In [2]:
#eliminamos bases de datos existentes
db = ['UC_database.sqlite', 'UoE_database.sqlite', 'Combined_database.sqlite']


for file in db:
    try:
        if os.path.exists(file):
            os.remove(file)
            print(f"Deleted database file: {file}")
        else:
            print(f"Database file not found: {file}")
    except Exception as e:
        print(f"Error deleting database file {file}: {e}")



Deleted database file: UC_database.sqlite
Deleted database file: UoE_database.sqlite
Deleted database file: Combined_database.sqlite


In [3]:
#base de datos UC

conn_uc = sqlite3.connect('UC_database.sqlite')  
c_uc = conn_uc.cursor() 


c_uc.execute('''
    CREATE TABLE IF NOT EXISTS Curso (
        idCurso INTEGER PRIMARY KEY,
        titulo VARCHAR(45) NOT NULL,
        creditos INTEGER NOT NULL
    )
''')
c_uc.execute('''
    CREATE TABLE IF NOT EXISTS Estudiante_has_Curso (
        Estudiante_idEstudiante INTEGER,
        Curso_idCurso INTEGER,
        PRIMARY KEY (Estudiante_idEstudiante, Curso_idCurso),
        FOREIGN KEY (Estudiante_idEstudiante) REFERENCES Estudiante(idEstudiante),
        FOREIGN KEY (Curso_idCurso) REFERENCES Curso(idCurso)
    )
''')
c_uc.execute('''
    CREATE TABLE  IF NOT EXISTS Estudiante (
        idEstudiante INTEGER PRIMARY KEY,
        nombre VARCHAR(25) NOT NULL,
        apellidos VARCHAR(85) NOT NULL,
        numMatricula VARCHAR(15),
        direccion VARCHAR(100),
        telefono VARCHAR(15),
        email VARCHAR(45)
    )
''')
c_uc.execute('''
    CREATE TABLE IF NOT EXISTS Profesor (
        idProfesor INTEGER PRIMARY KEY,
        nombre VARCHAR(25) NOT NULL,
        apellidos VARCHAR(85) NOT NULL,
        email VARCHAR(45)
    )
''')

c_uc.execute('''
    CREATE TABLE IF NOT EXISTS Profesor_has_Curso (
        Profesor_idProfesor INTEGER,
        Curso_idCurso INTEGER,
        PRIMARY KEY (Profesor_idProfesor, Curso_idCurso),
        FOREIGN KEY (Profesor_idProfesor) REFERENCES Profesor(idProfesor),
        FOREIGN KEY (Curso_idCurso) REFERENCES Curso(idCurso)
    )
''')
c_uc.execute('''
    CREATE TABLE IF NOT EXISTS EstudianteErasmus (
        idEstudianteErasmus INT,
        idEstudiante INT,
        universidadOrigen VARCHAR(45),
        PRIMARY KEY (idEstudiante, idEstudianteErasmus),
        FOREIGN KEY (idEstudiante) REFERENCES Estudiante(idEstudiante)
    )
''')
conn_uc.commit()
conn_uc.close()       

In [4]:
# Base de datos UoE

conn_uoe = sqlite3.connect('UoE_database.sqlite')
c_uoe = conn_uoe.cursor()


c_uoe.execute('''
    CREATE TABLE IF NOT EXISTS Program (
        idProgram INTEGER PRIMARY KEY,
        name VARCHAR(45) NOT NULL,
        ueFees INTEGER,
        overseasFees VARCHAR(45)
    )
''')
c_uoe.execute('''
    CREATE TABLE IF NOT EXISTS SportActivity (
        idSportActivity INTEGER PRIMARY KEY,
        name VARCHAR(45) NOT NULL,
        durationWeeks INTEGER NOT NULL,
        price INTEGER NOT NULL
    )
''')
c_uoe.execute('''
    CREATE TABLE IF NOT EXISTS Student (
        idStudent INTEGER PRIMARY KEY,
        name VARCHAR(155) NOT NULL,
        dateOfBirth DATE NOT NULL,
        nationality VARCHAR(45),
        isErasmus BOOLEAN,
        homeUniversity VARCHAR(45),
        hasScolarship BOOLEAN,
        Program_idProgram INTEGER,
        englishLevel CHAR(2),
        SportActivity_idSportActivity INTEGER,
        FOREIGN KEY (Program_idProgram) REFERENCES Program(idProgram),
        FOREIGN KEY (SportActivity_idSportActivity) REFERENCES SportActivity(idSportActivity)
    )
''')
c_uoe.execute('''
    CREATE TABLE IF NOT EXISTS Subject (
        idSubject INTEGER NOT NULL,
        name VARCHAR(45) NOT NULL,
        year INTEGER,
        Program_idProgram INTEGER,
        PRIMARY KEY(idSubject),
        FOREIGN KEY (Program_idProgram) REFERENCES Program(idProgram)
    )
''')
c_uoe.execute('''
    CREATE TABLE IF NOT EXISTS StudentMarks (
        Student_idStudent INTEGER,
        Subject_idSubject INTEGER,
        mark INTEGER NOT NULL,
        PRIMARY KEY (Student_idStudent, Subject_idSubject),
        FOREIGN KEY (Student_idStudent) REFERENCES Student(idStudent),
        FOREIGN KEY (Subject_idSubject) REFERENCES Subject(idSubject)
    )
''')
conn_uoe.commit()
conn_uoe.close()

Tarea 2: inserción de datos artificiales en las bases de datos

In [5]:
fake = Faker()
fakeEs = Faker("es_ES")

In [6]:
#uc
conn_uc = sqlite3.connect('UC_database.sqlite')
c_uc = conn_uc.cursor()


for idProfesor in range(10):   
    nombre = fakeEs.first_name()
    apellidos = fakeEs.last_name()
    email = fakeEs.email()

    c_uc.execute('''
        INSERT INTO Profesor (idProfesor, nombre, apellidos, email) VALUES (?, ?, ?, ?)''',
        (idProfesor,nombre, apellidos, email))

#tabla estudiantes: insertamos primero 10 alumnos erasmus
for idEstudiante in range(10): 
    nombre = fake.first_name()
    apellidos = fake.last_name()
    numMatricula = fake.random_number(digits=8)
    direccion = fake.address()
    telefono = fake.phone_number()
    email = fake.email()

    c_uc.execute('''
        INSERT INTO Estudiante (idEstudiante,nombre, apellidos, numMatricula, direccion, telefono, email)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (idEstudiante, nombre, apellidos, numMatricula, direccion, telefono, email))

#tabla estudiantes: insertamos primero 90 alumnos españoles
for idEstudiante in range(11,100): 
    nombre = fakeEs.first_name()
    apellidos = fakeEs.last_name()
    numMatricula = fakeEs.random_number(digits=8)
    direccion = fakeEs.address()
    telefono = fakeEs.phone_number()
    email = fakeEs.email()

    c_uc.execute('''
        INSERT INTO Estudiante (idEstudiante,nombre, apellidos, numMatricula, direccion, telefono, email)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (idEstudiante, nombre, apellidos, numMatricula, direccion, telefono, email))



for idCurso in range(20):  
    titulo = fakeEs.word()
    creditos = fakeEs.random_int(min=2, max=6)

    c_uc.execute('''
        INSERT INTO Curso (idCurso,titulo, creditos)
        VALUES (?, ?,?)
    ''', (idCurso,titulo, creditos))


for estudiante_idEstudiante in range(100):   
    Curso_idCurso = fakeEs.random_int(min=0, max=19)
    c_uc.execute('''
        INSERT INTO Estudiante_has_Curso (Curso_idCurso, Estudiante_idEstudiante)
        VALUES (?, ?)
    ''', (Curso_idCurso, estudiante_idEstudiante))

for Profesor_idProfesor in range(10):   
    Curso_idCurso = fakeEs.random_int(min=0, max=19)
    c_uc.execute('''
        INSERT INTO Profesor_has_Curso (Profesor_idProfesor, Curso_idCurso)
            VALUES (?, ?)
        ''', (Profesor_idProfesor, Curso_idCurso))


for idEstudiante in range(10):  
    universidadOrigen = fake.city()
    idEstudianteErasmus = idEstudiante
    c_uc.execute('''
        INSERT INTO EstudianteErasmus (idEstudianteErasmus, idEstudiante, universidadOrigen)
        VALUES (?, ?,?)
    ''', (idEstudianteErasmus, idEstudiante, universidadOrigen))


conn_uc.commit()
conn_uc.close()

In [7]:
#UoE
conn_uoe = sqlite3.connect('UoE_database.sqlite')
c_uoe = conn_uoe.cursor()


for idStudent in range(100): 
    name = fake.name()
    dateOfBirth = fake.date()
    nationality = fake.country()
    isErasmus = random.choice([True, False])
    if isErasmus:
        homeUniversity = fake.city()
    else:
        homeUniversity = "Edinburg"
    hasScolarship = random.choice([True, False])
    Program_idProgram = random.randint(0, 29)
    englishLevel = random.choice(["a1","a2","b1","b2","c1","c2"])
    SportActivity_idSportActivity = random.randint(0, 9)

    c_uoe.execute('''
        INSERT INTO Student (idStudent,name,dateOfBirth,nationality,
        isErasmus,homeUniversity,hasScolarship,Program_idProgram,
        englishLevel,SportActivity_idSportActivity)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?,?,?)
    ''', (idStudent,name,dateOfBirth,nationality,
        isErasmus,homeUniversity,hasScolarship,Program_idProgram,
        englishLevel,SportActivity_idSportActivity))


for idSubject in range(20):  
    name = fake.word()
    year = fake.random_int(min=1, max=4)
    Program_idProgram = random.randint(0, 29)

    c_uoe.execute('''
        INSERT INTO Subject (idSubject,name,year,Program_idProgram)
        VALUES (?, ?,?,?)
    ''',  (idSubject,name,year,Program_idProgram))


for Student_idStudent in range(100):   
    Subject_idSubject = random.randint(0, 19)
    mark = random.randint(0, 10)
    c_uoe.execute('''
        INSERT INTO StudentMarks (Student_idStudent,Subject_idSubject,mark)
        VALUES (?, ?,?)
    ''', (Student_idStudent,Subject_idSubject,mark))



for idSportActivity in range(10):   
    name = fake.word()
    durationWeeks = random.randint(1, 36)
    price = random.randint(0, 100)
    c_uoe.execute('''
        INSERT INTO SportActivity (idSportActivity,name,durationWeeks,price)
            VALUES (?, ?,?,?)
        ''', (idSportActivity,name,durationWeeks,price))


for idProgram in range(30):   
    name = fake.word()
    ueFees = random.randint(0, 50)
    overseasFees = random.randint(0, 100)
    c_uoe.execute('''
        INSERT INTO Program (idProgram,name,ueFees,overseasFees)
            VALUES (?, ?, ?, ?)
        ''', (idProgram,name,ueFees,overseasFees))

conn_uoe.commit()
conn_uoe.close()

Tarea 3: creación de la base de datos integrada. 

Teniendo en cuenta la estructura de ambas bases de datos, las tablas "curso", "estudiante_has_curso" y "estudiante" de la base de UC se mantienen y amplian con los campos de "Subject","EstudentMarks" y "Student" de la base de UoE. A partir de estas tablas, se agregan otras de forma que no se pierda información de ninguna de las bases de datos originales. 

In [8]:
#base de datos integrada

conn_comb = sqlite3.connect('Combined_database.sqlite')  
c_comb = conn_comb.cursor() 


c_comb.execute('''
    CREATE TABLE IF NOT EXISTS Curso (
        idUni INTEGER,
        idCurso INTEGER,
        titulo VARCHAR(45) NOT NULL,
        anno INTEGER,
        creditos INTEGER,
        Programa_idPrograma INTEGER,
        PRIMARY KEY(idUni,idCurso)
        FOREIGN KEY (Programa_idPrograma) REFERENCES Programa(idPrograma))''')
c_comb.execute('''
    CREATE TABLE IF NOT EXISTS Estudiante_has_Curso (
        idUni INTEGER,
        Estudiante_idEstudiante INTEGER,
        Curso_idCurso INTEGER,
        nota INTEGER,
        PRIMARY KEY (idUni, Estudiante_idEstudiante, Curso_idCurso),
        FOREIGN KEY (Estudiante_idEstudiante) REFERENCES Estudiante(idEstudiante),
        FOREIGN KEY (Curso_idCurso) REFERENCES Curso(idCurso)
    )
''')
c_comb.execute('''
    CREATE TABLE  IF NOT EXISTS Estudiante (
        idUni INTEGER,
        idEstudiante INTEGER,
        fechaDeNacimiento DATE,
        nacionalidad VARCHAR(85),
        nombre VARCHAR(25) NOT NULL,
        apellidos VARCHAR(85),
        tieneBeca BOOLEAN,
        numMatricula VARCHAR(15),
        direccion VARCHAR(100),
        telefono VARCHAR(15),
        email VARCHAR(45),
        Programa_idPrograma INTEGER,
        nivelIngles CHAR(2),
        Deporte_idDeporte INTEGER,
        PRIMARY KEY (idUni ,idEstudiante),
        FOREIGN KEY (Programa_idPrograma) REFERENCES Programa(idPrograma),
        FOREIGN KEY (Deporte_idDeporte) REFERENCES Deporte(idDeporte)        
    )
''')

#tabla solo usada para UC, por lo que no hace falta distinguir entre
#profesores de varias universidades. idProfesor se refiere al profesor
#cuyo idUni seria el id de UC.
c_comb.execute('''
    CREATE TABLE IF NOT EXISTS Profesor (
        idProfesor INTEGER PRIMARY KEY,
        nombre VARCHAR(25) NOT NULL,
        apellidos VARCHAR(85) NOT NULL,
        email VARCHAR(45)
    )
''')

#de la misma forma, no distinguimos los cursos asociados a profesores 
#son solo de la UC por lo que no incluimos idUni
c_comb.execute('''
    CREATE TABLE IF NOT EXISTS Profesor_has_Curso (
        Profesor_idProfesor INTEGER,
        Curso_idCurso INTEGER,
        PRIMARY KEY (Profesor_idProfesor, Curso_idCurso),
        FOREIGN KEY (Profesor_idProfesor) REFERENCES Profesor(idProfesor),
        FOREIGN KEY (Curso_idCurso) REFERENCES Curso(idCurso)
    )
''')
c_comb.execute('''
    CREATE TABLE IF NOT EXISTS EstudianteErasmus (
        idUni INT,
        idEstudianteErasmus INT,
        idEstudiante INT,
        universidadOrigen VARCHAR(45),
        PRIMARY KEY (idUni,idEstudiante, idEstudianteErasmus),
        FOREIGN KEY (idEstudiante) REFERENCES Estudiante(idEstudiante)
    )
''')
#de la misma forma, el idPrograma solo corresponde a programas con idUni
# el de UoE
c_comb.execute('''
    CREATE TABLE IF NOT EXISTS Programa (
        idPrograma INTEGER PRIMARY KEY,
        nombre VARCHAR(45) NOT NULL,
        TarifaUE INTEGER,
        TarifaNoUE VARCHAR(45)
    )
''')
c_comb.execute('''
    CREATE TABLE IF NOT EXISTS Deporte (
        idDeporte INTEGER PRIMARY KEY,
        nombre VARCHAR(45) NOT NULL,
        semanas INTEGER NOT NULL,
        precio INTEGER NOT NULL
    )
''')

conn_comb.commit()
conn_comb.close()   

Tarea 4: extracción, transformación y carga de los datos originales en  la base de datos integrada

In [9]:
#Extracción

def extraccion(database_file):
    # extrae las tablas de una base de datos en forma de un diccionario
  
    conn = sqlite3.connect(database_file)
    c = conn.cursor()

    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = c.fetchall()
    
    
    dictExtr = {}


    for table in tables:
        table_name = table[0]
        c.execute(f"SELECT * FROM {table_name};")
        table_data = c.fetchall()
        dictExtr[table_name] = table_data

    # Cerrar la conexión
    conn.close()

    return dictExtr

In [10]:
dictUc = extraccion('UC_database.sqlite')

In [11]:
dictUoE = extraccion('UoE_database.sqlite')

In [12]:
#Transformación e inserción
#UC a combined

conn = sqlite3.connect('Combined_database.sqlite')
c = conn.cursor()
idUni = [1]


#Estudiante
for elemento in dictUc["Estudiante"]:  
    fila = list(elemento)
    #insertamos None en campos no definidos para UC
    fila[0:0] =  idUni
    fila[2:2] =  [None] * 2
    fila[6:6] =  [None] * 1
    fila[11:11] =  [None] * 3
    fila = tuple(fila)
    c.execute('''
        INSERT INTO Estudiante (idUni,idEstudiante,fechaDeNacimiento,
        nacionalidad,nombre,apellidos,tieneBeca,numMatricula,direccion,
        telefono,email,Programa_idPrograma,nivelIngles,Deporte_idDeporte)
        VALUES (?, ?,?,?,?, ?,?,?,?, ?,?,?,?,?)
    ''', fila)

#Curso
for elemento in dictUc["Curso"]:
    fila = list(elemento)
    #insertamos None en campos no definidos para UC
    fila[0:0] =  idUni
    fila[3:3] =  [None] * 1
    fila[5:5] =  [None] * 1
    fila = tuple(fila)
    c.execute('''
        INSERT INTO Curso (idUni,idCurso,titulo,anno,creditos,Programa_idPrograma)
        VALUES (?,?,?,?,?,?)
    ''', fila)

#Estudiante_has_Curso
for elemento in dictUc["Estudiante_has_Curso"]:  
    fila = list(elemento)
    #insertamos None en campos no definidos para UC
    fila[0:0] =  idUni
    fila[3:3] =  [None] * 1
    fila = tuple(fila)
    c.execute('''
        INSERT INTO Estudiante_has_Curso (idUni,Estudiante_idEstudiante,Curso_idCurso,nota)
        VALUES (?, ?,?,?)
    ''', fila)
#Profesor
for elemento in dictUc["Profesor"]:   
    c.execute('''
        INSERT INTO Profesor (idProfesor,nombre,apellidos,email)
        VALUES (?, ?,?,?)
    ''', elemento)
#Profesor_has_Curso
for elemento in dictUc["Profesor_has_Curso"]:   
    c.execute('''
        INSERT INTO Profesor_has_Curso (Profesor_idProfesor,Curso_idCurso)
        VALUES (?, ?)
    ''', elemento)
#EstudianteErasmus
for elemento in dictUc["EstudianteErasmus"]:  
    fila = list(elemento)
    #insertamos None en campos no definidos para UC
    fila[0:0] =  idUni
    fila = tuple(fila)
    c.execute('''
        INSERT INTO EstudianteErasmus (idUni,idEstudianteErasmus,idEstudiante,universidadOrigen)
        VALUES (?,?,?,?)
    ''', fila)

conn.commit()
conn.close()

In [13]:
#Transformación e inserción
#UoE a combined

conn = sqlite3.connect('Combined_database.sqlite')
c = conn.cursor()
idUni = [2]


#Estudiante
for elemento in dictUoE["Student"]:  
    fila = list(elemento)
    #las filas de la tabla Student de UoE tienen loos campos
    #en otro orden por lo que hay que hacer un cambio antes
    #de volcarlos a la base de datos combinada
    #insertamos None en campos no definidos para UoE
    
    fila[0:0] =  idUni
    #fechaNacimiento en tercerlugar
    nacionalidad = fila[4]
    fila[4:4] = [fila[2]]
    fila[2] = fila[3]
    fila.pop(3)
    #nacionalidad en cuarto lugar
    fila[3:3] = [nacionalidad]
    fila.pop(5)
    
    #fila[4] = fila[3]
    #fila.pop(4)
    
    fila[5:5]= [None]
    #tiene beca
    fila[6] = fila[8]
    fila.pop(8)
    #NumMat
    fila[7]= None
    #otros
    fila[8:8]=[None] * 3
    
    
    fila = tuple(fila)
    c.execute('''
        INSERT INTO Estudiante (idUni,idEstudiante,fechaDeNacimiento,
        nacionalidad,nombre,apellidos,tieneBeca,numMatricula,direccion,
        telefono,email,Programa_idPrograma,nivelIngles,Deporte_idDeporte)
        VALUES (?, ?,?,?,?, ?,?,?,?, ?,?,?,?,?)
    ''', fila)

#Curso
for elemento in dictUoE["Subject"]:
    fila = list(elemento)
    #insertamos None en campos no definidos para UC
    fila[0:0] =  idUni
    fila[4:4] =  [None] 
    fila = tuple(fila)
    c.execute('''
        INSERT INTO Curso (idUni,idCurso,titulo,anno,creditos,Programa_idPrograma)
        VALUES (?,?,?,?,?,?)
    ''', fila)

#Estudiante_has_Curso
for elemento in dictUoE["StudentMarks"]:  
    fila = list(elemento)
    #insertamos None en campos no definidos para UC
    fila[0:0] =  idUni
    fila = tuple(fila)
    c.execute('''
        INSERT INTO Estudiante_has_Curso (idUni,Estudiante_idEstudiante,Curso_idCurso,nota)
        VALUES (?, ?,?,?)
    ''', fila)


#Programa
for elemento in dictUoE["Program"]:   
    c.execute('''
        INSERT INTO Programa (idPrograma,nombre,TarifaUE,TarifaNoUE)
        VALUES (?,?,?,?)
    ''', elemento)
i = 0
for elemento in dictUoE["Student"]:  
    #recorremos la tabla student para ver cuales son erasmus con el campo isErasmus
    if elemento[4]:
        fila = list(elemento)
        fila = fila[:4]
        fila[0] =  2
        #como idEstudianteErasmus forma parte de la PK de la tabla, añadimos un valor
        #por orden de inserción
        fila[1] = i
        i = i+1
        #idEstudiante
        fila[2] = elemento[0]
        #universidadOrigen
        fila[3] = elemento[5]
        fila = tuple(fila)
        c.execute('''
            INSERT INTO EstudianteErasmus (idUni,idEstudianteErasmus,idEstudiante,universidadOrigen)
            VALUES (?,?,?,?)
        ''', fila)
for elemento in dictUoE["SportActivity"]:   
    c.execute('''
        INSERT INTO Deporte (idDeporte,nombre,semanas,precio)
        VALUES (?,?,?,?)
    ''', elemento)

conn.commit()
conn.close()

In [14]:
#consulta de prueba
conn = sqlite3.connect('Combined_database.sqlite')
c = conn.cursor()
print("deportes")
deporte = c.execute('''SELECT * from Deporte''')
for d in deporte:
    print(d)

print("estudiantes")
estudiantes = c.execute('''SELECT * from Estudiante''')
for e in estudiantes:
    print(e)

conn.commit()
conn.close()

deportes
(0, 'art', 6, 15)
(1, 'something', 13, 67)
(2, 'trial', 9, 3)
(3, 'worry', 17, 36)
(4, 'office', 26, 24)
(5, 'message', 5, 81)
(6, 'rich', 5, 22)
(7, 'special', 26, 40)
(8, 'choice', 20, 19)
(9, 'teacher', 3, 37)
estudiantes
(1, 0, None, None, 'Christopher', 'Stewart', None, '98078855', '292 Jerry Wells Suite 858\nCharleschester, MH 44003', '001-349-224-7832', 'edwardsgregory@example.com', None, None, None)
(1, 1, None, None, 'Michelle', 'Pearson', None, '75668312', '27661 Buckley Wall Suite 673\nJeffreybury, ME 73423', '255.283.2126x06280', 'ruthbrown@example.com', None, None, None)
(1, 2, None, None, 'Ryan', 'Ross', None, '65888482', '35746 Charles Islands Suite 676\nMartinezchester, AZ 20852', '6524035579', 'otaylor@example.net', None, None, None)
(1, 3, None, None, 'Jessica', 'Calhoun', None, '63860870', '80449 Wong Harbors\nNorth Matthewtown, HI 17764', '(931)690-8749x304', 'romerodavid@example.com', None, None, None)
(1, 4, None, None, 'Bailey', 'Carter', None, '17811630